In [8]:
import Pkg
Pkg.activate("../")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner\_wip\TCVAR`


In [70]:
include("../src/TCVAR.jl")

Main.TCVAR

In [71]:
using .TCVAR
using DataFrames, XLSX, TimeSeries
using Statistics
using LinearAlgebra

In [17]:
df = DataFrame(XLSX.readtable("../../../data/usa_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

292×12 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1951-12-01 │  0.02513 │   104.098 │       23.77 │  1.41333 │   2.44 │ 12.527 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       24.37 │     1.43 │    2.4 │ 12.242 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       24.96 │     1.45 │   2.34 │ 12.669 ⋯
│ 1952-09-01 │  0.02685 │   104.468 │       24.54 │  1.43667 │   2.36 │ 12.131 ⋯
│ 1952-12-01 │  0.02547 │   106.402 │       26.57 │     1.41 │    2.4 │ 13.010 ⋯
│ 1953-03-01 │  0.02735 │   105.373 │       25.29 │  1.41333 │   2.43 │ 12.163 ⋯
│ 1953-06-01 │  0.02987 │    103.86 │       24.14 │     1.42 │   2.51 │ 11.750 ⋯
│ 1953-09-01 │  0.02636 │    107.81 │       23.35 │     1.43 │   2.55 │ 11.391 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [41]:
cpi = data_source[:CPI] ./ lag(data_source[:CPI], 4) .- 1
sr = data_source[:T_Bill]

data = merge(cpi, sr) .* 100

presample, data = to(data, Date(1957, 11, 30)), from(data, Date(1957, 12,01))


(20×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 1952-12-01 to 1957-09-01, 268×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 1957-12-01 to 2024-07-01)

In [52]:
presample_mean = Statistics.mean(presample)
display(presample_mean)

presample_variance = var(presample)
display(presample_variance)
display(presample_variance .^ 2)

1×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 1957-09-01 to 1957-09-01
┌────────────┬─────────┬────────┐
│            │ CPI_CPI │ T_Bill │
├────────────┼─────────┼────────┤
│ 1957-09-01 │ 1.07596 │  2.071 │
└────────────┴─────────┴────────┘

1×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 1957-09-01 to 1957-09-01
┌────────────┬─────────┬──────────┐
│            │ CPI_CPI │ T_Bill   │
├────────────┼─────────┼──────────┤
│ 1957-09-01 │   1.849 │ 0.706494 │
└────────────┴─────────┴──────────┘

1×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 1957-09-01 to 1957-09-01
┌────────────┬─────────┬──────────┐
│            │ CPI_CPI │ T_Bill   │
├────────────┼─────────┼──────────┤
│ 1957-09-01 │ 3.41879 │ 0.499133 │
└────────────┴─────────┴──────────┘

In [66]:
n = 2 #number of observatin variables

priors = (
        initial_trend_mean = values(presample_mean)[1,:],
        initial_cycle_mean = zeros(2),
        initial_trend_covariance = diagm([1., 1.]),
        trend_covariance_df = 100,
        trend_covariance_mean = diagm([2. /10, 1. /20].^2),
        cycle_coeff_mean = zeros(2, 2),
        cycle_coeff_shrinkage_param = .2,
        cycle_covariance_mean = diagm(values(presample_variance)[1,:]), #diagm(presample_variance), 
        cycle_covariance_df = n+2
        )


(initial_trend_mean = [1.0759553361960457, 2.071], initial_cycle_mean = [0.0, 0.0], initial_trend_covariance = [1.0 0.0; 0.0 1.0], trend_covariance_df = 100, trend_covariance_mean = [0.04000000000000001 0.0; 0.0 0.0025000000000000005], cycle_coeff_mean = [0.0 0.0; 0.0 0.0], cycle_coeff_shrinkage_param = 0.2, cycle_covariance_mean = [1.8489975236133918 0.0; 0.0 0.7064936842105262], cycle_covariance_df = 4)

In [67]:
priors.initial_trend_mean

2-element Vector{Float64}:
 1.0759553361960457
 2.071

In [72]:
trend_states_samples, cycle_states_samples, trend_covariance_samples, betas_samples, sigmas_samples = TCVAR.gibs_sampler(values(data), priors; burnin = 5_000, n_samples = 5_000, thin=5)

trend_states_mean, trend_states_lower, trend_states_upper = compute_posterior_statistics(trend_states_samples, credible_level=0.95)  
cycle_states_mean, cycle_states_lower, cycle_states_upper = compute_posterior_statistics(cycle_states_samples, credible_level=0.95) 

DimensionMismatch: DimensionMismatch: a has size (2, 2), b has size (4, 4), mismatch at dim 1